# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

F. Walter  ->  F. Walter  |  ['F. Walter']
K. Lee  ->  K. Lee  |  ['K. Lee']
M. Bergemann  ->  M. Bergemann  |  ['M. Bergemann']
E. Magg  ->  E. Magg  |  ['E. Magg']
Arxiv has 76 new papers today
          3 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2502.08984


extracting tarball to tmp_2502.08984... done.


F. Walter  ->  F. Walter  |  ['F. Walter']


/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure empty.eps
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:707: LatexWarning: Could not find graphic \includegraphics[width=\hsize]{empty.eps}
  warnings.warn(LatexWarning(f"Could not find graphic {k}"))
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure empty.eps
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:707: LatexWarning: Could not find graphic \includegraphics[angle=-90,width=3cm]{empty.eps}
  warnings.warn(LatexWarning(f"Could not find graphic {k}"))
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/arxiv_on_d

...

extracting tarball to tmp_2502.09275...

 done.
Retrieving document from  https://arxiv.org/e-print/2502.09388
extracting tarball to tmp_2502.09388...

 done.


M. Bergemann  ->  M. Bergemann  |  ['M. Bergemann']
E. Magg  ->  E. Magg  |  ['E. Magg']


Issues with the citations
list index out of range


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2502.08984-b31b1b.svg)](https://arxiv.org/abs/2502.08984) | **The [NII] 205 $μ$m line emission from high-z SMGs and QSOs**  |
|| S. Kolupuri, et al. -- incl., <mark>F. Walter</mark> |
|*Appeared on*| *2025-02-14*|
|*Comments*| *14 pages, 11 figures, accepted for publication in A&A*|
|**Abstract**|            We present [NII] 205 $\mu$m fine structure line observations of three submillimeter galaxies (SMGs) and three quasar host galaxies at 4$\lesssim$z$\lesssim$6 using the Institut de radioastronomie millimetrique (IRAM) interferometer. The [NII] emission is detected in three sources, and we report detections of the underlying dust continuum emission in all sources. The observed [NII]-to-infrared luminosity ratio spans at least 0.5 dex for our sources. Comparing our estimates with sources detected in the [NII] 205 $\mu$m at similar redshifts shows that the overall [NII]-to-IR luminosity ratio spans over a dex in magnitude from L$_{[NII]}$/L$_{IR}$ ~ 10$^{-4}$ - 10$^{-5}$ and follows the trend of the so-called [NII] fine structure line deficit observed in (ultra)-luminous infrared galaxies in the local Universe. The [CII]-to-[NII] luminosity ratio is >10 for most of our sources, indicating that the bulk of the [CII] 158 $\mu$m line emission (f([CII]$^{PDR}$)>75%) arises from the neutral medium. From our analysis, we do not find significant differences in the [NII] 205 $\mu$m emission and the respective ratios between SMGs and QSOs, suggesting a negligible contribution to the boosting of [NII] 205 $\mu$m emission due to the active galactic nucleus (AGN) photoionization. Future investigations involving other fine structure lines and optical diagnostics will provide further insight into a suite of ionized medium properties and reveal the diversity between AGN and non-AGN environments.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2502.09388-b31b1b.svg)](https://arxiv.org/abs/2502.09388) | **Performance of the Stellar Abundances and atmospheric Parameters Pipeline adapted for M dwarfs I. Atmospheric parameters from the spectroscopic module**  |
|| T. Olander, et al. -- incl., <mark>M. Bergemann</mark>, <mark>E. Magg</mark> |
|*Appeared on*| *2025-02-14*|
|*Comments*| *Accepted in A&A*|
|**Abstract**|            M dwarfs are important targets in the search for Earth-like exoplanets due to their small masses and low luminosities. Several ongoing and upcoming space missions are targeting M dwarfs for this reason, and the ESA PLATO mission is one of these. In order to fully characterise a planetary system the properties of the host star must be known. For M dwarfs we can derive effective temperature, surface gravity, metallicity, and abundances of various elements from spectroscopic observations in combination with photometric data. The Stellar Abundances and atmospheric Parameters Pipeline (SAPP) has been developed as a prototype for one of the stellar science softwares within the PLATO consortium, it is aimed at FGK stars. We have modified it to be able to analyse the M dwarf among the PLATO targets. The current version of the pipeline for M dwarfs mostly relies on spectroscopic observations. The data processing is based on the machine learning algorithm The Payne and fits a grid of model spectra to an observed spectrum to derive effective temperature and metallicity. We use spectra in the H-band, as the near-infrared region is beneficial for M dwarfs. A method based on synthetic spectra was developed for the continuum normalisation of the spectra, taking into account the pseudo-continuum formed by numerous lines of the water molecule. Photometry is used to constrain the surface gravity. We tested the modified SAPP on spectra of M dwarfs from the APOGEE survey. Our validation sample of 26 stars includes stars with interferometric observations and binaries. We found a good agreement between our values and reference values from a range of studies. The overall uncertainties in the derived effective temperature, surface gravity, and metallicity is 100 K, 0.1 dex, and 0.15 dex, respectively. We find that the modified SAPP performs well on M dwarfs and identify possible areas of future development.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2502.09275-b31b1b.svg)](https://arxiv.org/abs/2502.09275) | **Chinese Pulsar Timing Array upper limits on microhertz gravitational waves from supermassive black-hole binaries using PSR J1713+0747 FAST data**  |
|| R. N. Caballero, et al. -- incl., <mark>K. Lee</mark> |
|*Appeared on*| *2025-02-14*|
|*Comments*| *Accepted by Research in Astronomy and Astrophysics (RAA)*|
|**Abstract**|            We derive the gravitational-wave (GW) strain upper limits from resolvable supermassive black-hole binaries using the data from the Five-hundred-meter Aperture Spherical radio Telescope (FAST), in the context of the Chinese Pulsar Timing Array project. We focus on circular orbits in the $\mu$Hz GW frequency band between $10^{-7}$ and $3\times10^{-6}$ Hz. This frequency band is higher than the traditional pulsar timing array band and is less explored. We used the data of the millisecond pulsar PSR J1713+5307 observed between August 2019 and April 2021. A dense observation campaign was carried out in September 2020 to allow for the $\mu$Hz band coverage. Our sky-average continuous source upper limit at the 95% confidence level at 1$\mu$Hz is 1.26$\times10^{-12}$, while the same limit in the direction of the pulsar is 4.77$\times10^{-13}$.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['', '', '', '', '', '', '', 'tmp_2502.08984/./new_nll_ir_with_my_IR.png']
file not found 
file not found 
file not found 
file not found 
file not found 
file not found 
file not found 
copying  tmp_2502.08984/./new_nll_ir_with_my_IR.png to _build/html/
exported in  _build/html/2502.08984.md
    + _build/html/
    + _build/html/
    + _build/html/
    + _build/html/
    + _build/html/
    + _build/html/
    + _build/html/
    + _build/html/tmp_2502.08984/./new_nll_ir_with_my_IR.png
found figures ['tmp_2502.09388/./Figures/Teff_spec201222.png', 'tmp_2502.09388/./Figures/logg_spec201222.png', 'tmp_2502.09388/./Figures/met_spec201222.png', 'tmp_2502.09388/./Figures/GJ880_spectra201222.png', 'tmp_2502.09388/./Figures/Teff_interferometry201222.png', 'tmp_2502.09388/./Figures/logg_interferometry201222.png']
copying  tmp_2502.09388/./Figures/Teff_spec201222.png to _build/html/
copying  tmp_2502.09388/./Figures/logg_spec201222.png to _build/html/
copying  tmp_2502.09388/./Figure

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$</div>



<div id="title">

# The [NII] $205  \mu m$ line emission from high-z SMGs and QSOs

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2502.08984-b31b1b.svg)](https://arxiv.org/abs/2502.08984)<mark>Appeared on: 2025-02-14</mark> -  _14 pages, 11 figures, accepted for publication in A&A_

</div>
<div id="authors">

S. Kolupuri, et al. -- incl., <mark>F. Walter</mark>

</div>
<div id="abstract">

**Abstract:** We present [ NII ] 205 $\mu$ m fine structure line observations of three submillimeter galaxies (SMGs) and three quasar host galaxies at 4 $\lesssim$ z $\lesssim$ 6 using the Institut de radioastronomie millimétrique (IRAM) interferometer. The [ NII ] emission is detected in three sources, and we report detections of the underlying dust continuum emission in all sources. The observed [ NII ] -to-infrared luminosity ratio spans at least 0.5 dex for our sources. Comparing our estimates with sources detected in the [ NII ] 205 $\mu$ m at similar redshifts shows that the overall [ NII ] -to-IR luminosity ratio spans over a dex in magnitude from L $_{[NII]}$ /L $_{IR}$ $\sim$ 10 $^{-4}$ -- 10 $^{-5}$ and follows the trend of the so-called [ NII ] fine structure line deficit observed in (ultra)-luminous infrared galaxies in the local Universe. The [ CII ] -to- [ NII ] luminosity ratio is >10 for most of our sources, indicating that the bulk of the [ CII ] 158 $\mu$ m line emission ( $*f*$ ( [ CII ] $^{PDR}$ ) >75 \% ) arises from the neutral medium. From our analysis, we do not find significant differences in the [ NII ] 205 $\mu$ m emission and the respective ratios between SMGs and QSOs, suggesting a negligible contribution to the boosting of [ NII ] 205 $\mu$ m emission due to the active galactic nucleus (AGN) photoionization. Future investigations involving other fine structure lines and optical diagnostics will provide further insight into a suite of ionized medium properties and reveal the diversity between AGN and non-AGN environments.

</div>

<div id="div_fig1">

<img src="" alt="Fig5.1" width="25%"/><img src="" alt="Fig5.2" width="25%"/><img src="" alt="Fig5.3" width="25%"/><img src="" alt="Fig5.4" width="25%"/>

**Figure 5. -** Vibrational stability equation of state
               $S_{\mathrm{vib}}(\lg e, \lg \rho)$.
               $>0$ means vibrational stability.
              Vibrational stability equation of state
               $S_{\mathrm{vib}}(\lg e, \lg \rho)$.
               $>0$ means vibrational stability.
              Nonlinear Model ResultsNonlinear Model ResultsSpectral types and photometry for stars in the
  region.Spectral types and photometry for stars in the
  region.List of nearby SNe used in this work.Summary for ISOCAM sources with mid-IR excess
(YSO candidates).Summary for ISOCAM sources with mid-IR excess
(YSO candidates). Sample stars with absolute magnitudecontinued. Sample stars with absolute magnitudecontinued.Shown in greyscale is a...Plotted above...Complexes characterisation.Line data and abundances ...Continued. (*FigVibStab*)

</div>
<div id="div_fig2">

<img src="" alt="Fig12.1" width="33%"/><img src="" alt="Fig12.2" width="33%"/><img src="" alt="Fig12.3" width="33%"/>

**Figure 12. -** Vibrational stability equation of state
               $S_{\mathrm{vib}}(\lg e, \lg \rho)$.
               $>0$ means vibrational stability.
              Nonlinear Model ResultsNonlinear Model ResultsSpectral types and photometry for stars in the
  region.Spectral types and photometry for stars in the
  region.List of nearby SNe used in this work.Summary for ISOCAM sources with mid-IR excess
(YSO candidates).Summary for ISOCAM sources with mid-IR excess
(YSO candidates). Sample stars with absolute magnitudecontinued. Sample stars with absolute magnitudecontinued.Shown in greyscale is a...Plotted above...Complexes characterisation.Line data and abundances ...Continued. (*FigVibStab*)

</div>
<div id="div_fig3">

<img src="tmp_2502.08984/./new_nll_ir_with_my_IR.png" alt="Fig6" width="100%"/>

**Figure 6. -** The [NII]-to-IR ratio as a function of IR luminosity for the sources in our sample. Our sample is illustrated as stars, following the same color scheme in Fig. 1. We also compile sources from the local Universe (green triangles) as well as at high redshifts (orange/red squares \& blue triangles) that have been detected in [NII] 205 $\mu$m emission. The green and blue bars represent the mean error bars for their respective data points. All luminosities are corrected for gravitational magnification. Our new measurements further populate the sparser high-end of the infrared luminosity regime of such sources.  (*fig:2*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2502.08984"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\plato}{\textsc{Plato}}$
$\newcommand{\gaia}{\textit{Gaia}}$
$\newcommand{\ag}{\ensuremath{A_G}\xspace}$
$\newcommand{\ebpminrp}{\ensuremath{E(G_{\rm BP} - G_{\rm RP})}\xspace}$
$\newcommand{\bpminrp}{\ensuremath{G_\mathrm{BP}-G_\mathrm{RP}}\xspace}$
$\newcommand{\gmag}{\ensuremath{G}}$
$\newcommand{\bpmag}{\ensuremath{G_\mathrm{BP}}}$
$\newcommand{\rpmag}{\ensuremath{G_\mathrm{RP}}}$
$\newcommand{\mg}{M_\gmag}$
$\newcommand{\diam}{\theta_{\rm LD}}$
$\newcommand{\marcs}{\texttt{MARCS}}$
$\newcommand{\phoenix}{\texttt{PHOENIX}}$
$\newcommand{\sme}{\texttt{SME}}$
$\newcommand{\pysme}{\texttt{PySME}}$
$\newcommand{\vald}{\texttt{VALD}}$
$\newcommand{\teff}{T_{\mathrm{eff}}}$
$\newcommand{\lgg}{\log{g}}$
$\newcommand{\lggu}{\log{g / \mathrm{cm s^{-2}}}}$
$\newcommand{\rv}{\varv_{\mathrm{r}}}$
$\newcommand{\vsini}{\varv \sin i}$
$\newcommand{\vbroad}{\varv_{\text{broad}}}$
$\newcommand{\vmic}{V_{\mathrm{turb}}}$
$\newcommand{\vmac}{\xi_{\text{mac}}}$
$\newcommand{\lggf}{\log{gf}}$
$\newcommand{\feh}{\mathrm{\left[Fe/H\right]}}$
$\newcommand{\afe}{\mathrm{\left[\alpha/Fe\right]}}$
$\newcommand{\xfe}[1]{\mathrm{\left[#1/Fe\right]}}$
$\newcommand{\xh}[1]{\mathrm{\left[#1/H\right]}}$
$\newcommand{\abrat}[2]{\mathrm{\left[#1/#2\right]}}$
$\newcommand{\lgeps}[1]{\log{\epsilon_{\mathrm{#1}}}}$
$\newcommand{\dex}{\mathrm{dex}}$
$\newcommand{\nm}{\mathrm{nm}}$
$\newcommand{\kelvin}{\mathrm{K}}$
$\newcommand{\kms}{\mathrm{km s^{-1}}}$
$\newcommand\natexlab{#1}$</div>



<div id="title">

# Performance of the _Stellar Abundances and atmospheric Parameters Pipeline_ adapted for M dwarfs: I. Atmospheric parameters from the spectroscopic module

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2502.09388-b31b1b.svg)](https://arxiv.org/abs/2502.09388)<mark>Appeared on: 2025-02-14</mark> -  _Accepted in A&A_

</div>
<div id="authors">

T. Olander, et al. -- incl., <mark>M. Bergemann</mark>, <mark>E. Magg</mark>

</div>
<div id="abstract">

**Abstract:** M dwarfs are important targets in the search for Earth-like exoplanets due to their small masses and low luminosities. Several ongoing and upcoming space missions are targeting M dwarfs for this reason, and the ESA $\plato$ mission is one of these. In order to fully characterise a planetary system the properties of the host star must be known. For M dwarfs we can derive effective temperature, surface gravity, metallicity, and abundances of various elements from spectroscopic observations in combination with photometric data. The Stellar Abundances and atmospheric Parameters Pipeline (SAPP) has been developed to serve as a prototype for one of the stellar science softwares within the $\plato$ consortium. The pipeline combines results from a spectroscopy, a photometry, an interferometry, and an asteroseismology module to derive stellar parameters for FGK-type stars. We have modified the pipeline to be able to analyse the M dwarf part of the $\plato$ target sample. The current version of the pipeline for M dwarfs mostly relies on spectroscopic observations. The module processing these data is based on the machine learning algorithm The Payne and fits a grid of model spectra to an observed spectrum to derive effective temperature and metallicity. We use spectra in the H-band, as the near-infrared region is beneficial for M dwarfs because there are fewer molecular lines and they are brighter in this wavelength region than in the optical. A method based on synthetic spectra was developed for the continuum normalisation of the spectra, taking into account the pseudo-continuum formed by numerous lines of the water molecule. Photometry is used to constrain the surface gravity. We tested the modified SAPP on spectra of M dwarfs from the APOGEE survey. Our validation sample of 26 stars includes stars with interferometric observations and binaries. We found a good agreement between our derived values and reference values from a range of previous studies. We estimate the overall uncertainties in the derived effective temperature, surface gravity, and metallicity to be 100 K, 0.1 dex, and 0.15 dex, respectively. We find that the modified SAPP performs well on M dwarfs and identify possible areas of future development that should lead to an improved precision of the derived stellar parameters.

</div>

<div id="div_fig1">

<img src="tmp_2502.09388/./Figures/Teff_spec201222.png" alt="Fig10.1" width="33%"/><img src="tmp_2502.09388/./Figures/logg_spec201222.png" alt="Fig10.2" width="33%"/><img src="tmp_2502.09388/./Figures/met_spec201222.png" alt="Fig10.3" width="33%"/>

**Figure 10. -** SAPP results compared with spectroscopic results from \citet{APOGEE_Sarmento2021,Pass2019,2015ApJMann,2020AAMaldonado,2022ApJSouto,2022MNRAS.516.3802C}.
   Values derived using the SAPP are shown on the vertical axis, and the literature values are shown on the horizontal axis.
   _Left:_ effective temperature.
   _Middle:_ surface gravity. The uncertainties for \citet{APOGEE_Sarmento2021} and \citet{2022ApJSouto} are represented at the bottom of the figure in grey. One star is located outside of the borders of the figure: LSPM J1204+1728S for which the SAPP value is 4.86 dex and \citet{APOGEE_Sarmento2021} obtained 5.31 dex.
   _Right:_ metallicity. The black dashed line in all panels corresponds to the 1:1 ratio.
    (*Fig:Spec_param*)

</div>
<div id="div_fig2">

<img src="tmp_2502.09388/./Figures/GJ880_spectra201222.png" alt="Fig8" width="100%"/>

**Figure 8. -** Normalised observed spectrum of the star GJ 880 as black dashed line, and best-fit model (synthetic spectrum predicted by the Payne's ANN for the parameters given in Table \ref{tab:Result_analysis}) as orange solid line. Grey shaded areas indicate the location of the line mask we used. Derived parameters for this star are $\teff$: 3649 K, $\lgg$: 4.8 dex, and $\feh$: 0.25 dex. (*Fig:GJ880_spec*)

</div>
<div id="div_fig3">

<img src="tmp_2502.09388/./Figures/Teff_interferometry201222.png" alt="Fig3.1" width="50%"/><img src="tmp_2502.09388/./Figures/logg_interferometry201222.png" alt="Fig3.2" width="50%"/>

**Figure 3. -** Comparing $\teff$(top) and $\lgg$(bottom) derived from the SAPP with corresponding parameters based on interferometric angular diameters \citep{Boyajian2012,Rabus2019}. The black dashed line in both figures corresponds to the 1:1 ratio and the grey dotted line in the top panel corresponds to a linear fit to the values from \citet{Boyajian2012}. We excluded the outlier GJ 725B for which the SAPP derived a $\teff$ of roughly 3550 K (leftmost blue square in top panel) from the linear fit, for reasons discussed in the text. We note that the K3 dwarf GJ 105A with $\teff\sim$4600 K is not shown in the figures. (*Fig:Interferometry*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2502.09388"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

334  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

7  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

3  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
